In [ ]:
import pandas as pd
import numpy as np
import arviz as az
import bambi as bmb
from tqdm.notebook import tqdm
import plotly.express as px
import sidetable
tqdm.pandas()

### Проведем симуляцию на основе toy problem из библиотеки space bandits

In [ ]:
from space_bandits.toy_problem import get_customer_nl, get_rewards_nl, get_cust_reward_nl

get_customer = get_customer_nl
get_rewards = get_rewards_nl
get_cust_reward = get_cust_reward_nl
optimal_choices = [0, 1, 2]

Будем записывать результаты каждого подохода к определению лучшей стратегии для минимизации потерть (regret)

In [ ]:
df_cmab_regrets = pd.DataFrame()
df_cmab_rewards = pd.DataFrame()

In [ ]:
n = 5000

### Random

Рандомно выберем ручки для пользователей

In [ ]:
cumulative_regret_list= []
cumulative_reward_list = []
total_regret = 0
cumulative_reward = 0
n_arms = 3
arm_indices = np.array(range(n_arms))
i = 0

In [ ]:
pbar = tqdm(desc = 'while loop', total = n)
while i != n:
    
    #generate a customer
    cust = get_customer()
    reward_vec = get_rewards(cust)
                    
        # Display the selected arm
    i += 1
    arm = np.random.choice(arm_indices)
    
    reward = reward_vec[arm]
    
    best_choice = optimal_choices[cust[0]]
    mx = reward_vec[best_choice]
    
    regret = mx - reward
    total_regret += regret
    cumulative_regret_list.append(total_regret)
    cumulative_reward += reward
    cumulative_reward_list.append(cumulative_reward)
    pbar.update(1)
            
pbar.close()
df_cmab_regrets['Random'] = cumulative_regret_list
df_cmab_rewards['Random'] = cumulative_reward_list

### UCB

Воспользуемся MAB алгоритмом UCB, который не учитывает контекст

In [ ]:
cumulative_regret_list= []
cumulative_reward_list = []
total_regret = 0
cumulative_reward = 0
n_arms = 3
arm_indices = np.array(range(n_arms))
i = 0
coef = 100
Q = np.zeros(n_arms)
uncertainty = np.zeros(n_arms)
N = np.zeros(n_arms)

In [ ]:
pbar = tqdm(desc = 'while loop', total = n)
while i != n:
    
    #generate a customer
    cust = get_customer()
    reward_vec = get_rewards(cust)
    context = np.array([cust[1]])
        
                    
        # Display the selected arm
    if any(N==0):
        arm = np.random.choice(arm_indices[N==0])
    else:
        uncertainty = np.sqrt(np.log(i) / N)
        arm = np.argmax(Q +  coef * uncertainty)
    i += 1
    
    reward = reward_vec[arm]
    
    N[arm] += 1
    Q[arm] += (1 / N[arm]) * (reward - Q[arm])
    
    best_choice = optimal_choices[cust[0]]
    mx = reward_vec[best_choice]

    regret = mx - reward
    total_regret += regret
    cumulative_regret_list.append(total_regret)
    cumulative_reward += reward
    cumulative_reward_list.append(cumulative_reward)
    pbar.update(1)
            
pbar.close()
df_cmab_regrets['UCB'] = cumulative_regret_list
df_cmab_rewards['UCB'] = cumulative_reward_list

### Spance bandits linear

Воспользуемся готовой реализацией линейного CMAB на сэмплировании Томпсона 

In [ ]:
from space_bandits import LinearBandits

In [ ]:
num_actions = 3 
num_features = 2
model = LinearBandits(num_actions, num_features)
X = []
y = []
a = []
total_regret = 0
cumulative_reward = 0
cumulative_regret_list= []
cumulative_reward_list = []
i = 0

In [ ]:
pbar = tqdm(desc = 'while loop', total = n)
while i != n:
    
    #generate a customer
    cust = get_customer()
    reward_vec = get_rewards(cust)
    #prepare features for model
    context = np.array([cust[1]])
        
                    
        # Display the selected arm
    if (i+1) < 500:
        arm = np.random.choice(arm_indices)
    else:
        arm = model.action(context)
    i += 1
    
    reward = reward_vec[arm]
    X.append(context.tolist()[0])
    y.append(reward)
    a.append(arm)
    
    best_choice = optimal_choices[cust[0]]
    mx = reward_vec[best_choice]
    regret = mx - reward
    total_regret += regret
    cumulative_regret_list.append(total_regret)
    cumulative_reward += reward
    cumulative_reward_list.append(cumulative_reward)
    pbar.update(1)
    #Update the models with the latest batch of data
    if i  % 500 == 0 and i  != n :
        print("Updating the models at i:", i )
        
        train_df = pd.DataFrame(X,
                            columns=["Age", 'ARPU'])
        train_df['reward'] = y
        train_df['arm'] = a
        print('Распределение ручек:')
        print(train_df.arm.value_counts())
        print(f"Текущий regret: {cumulative_regret_list[-1]}")
        
        for index, row in tqdm(train_df.iterrows(),total =train_df.shape[0]):
            context = row[["Age", 'ARPU']].values
            action = int(row['arm'])
            reward = float(row['reward'])
            model.update(context, action, reward)
        
        X = []
        y = []
        a = []
            
pbar.close()
df_cmab_regrets['SB_linear'] = cumulative_regret_list
df_cmab_rewards['SB_linear'] = cumulative_reward_list

### Space bandits Neural

Воспользуемся готовой реализацией нелинейного CMAB на сэмплировании Томпсона и нейронной сети

In [ ]:
from space_bandits import NeuralBandits

In [ ]:
num_actions = 3 
num_features = 2
model = NeuralBandits(num_actions, num_features, layer_sizes=[32,10], initial_pulls=100, initial_lr=0.01,verbose=False,show_training=False)
X = []
y = []
a = []
total_regret = 0
cumulative_reward = 0
cumulative_regret_list= []
cumulative_reward_list = []
i = 0

In [ ]:
pbar = tqdm(desc = 'while loop', total = n)
while i != n:
    
    #generate a customer
    cust = get_customer()
    reward_vec = get_rewards(cust)
    #prepare features for model
    context = np.array([cust[1]])
        
                    
        # Display the selected arm
    if (i+1) < 500:
        #arm = model.action(context)
        arm = np.random.choice(arm_indices)
    else:
        arm = model.action(context)
    i += 1
    
    reward = reward_vec[arm]
    X.append(context.tolist()[0])
    y.append(reward)
    a.append(arm)
    
    best_choice = optimal_choices[cust[0]]
    mx = reward_vec[best_choice]
    regret = mx - reward
    total_regret += regret
    cumulative_regret_list.append(total_regret)
    cumulative_reward += reward
    cumulative_reward_list.append(cumulative_reward)
    pbar.update(1)
    #Update the models with the latest batch of data
    if i  % 500 == 0 and i  != n :
        print("Updating the models at i:", i )
        
        train_df = pd.DataFrame(X,
                            columns=["Age", 'ARPU'])
        train_df['reward'] = y
        train_df['arm'] = a
        print('Распределение ручек:')
        print(train_df.arm.value_counts())
        print(f"Текущий regret: {cumulative_regret_list[-1]}")
        
        for index, row in tqdm(train_df.iterrows(),total =train_df.shape[0]):
            context = row[["Age", 'ARPU']].values
            action = int(row['arm'])
            reward = float(row['reward'])
            model.update(context, action, reward)
        
        X = []
        y = []
        a = []
            
pbar.close()
df_cmab_regrets['SB_neural_linear'] = cumulative_regret_list
df_cmab_rewards['SB_neural_linear'] = cumulative_reward_list

### PYMC linear CMAB

Воспользуемся нашей реализацией линейного CMAB на сэмплировании Томпсона 

Определим функции для бандита

Функция отвечающая за инициализацию априорного распределения факторов

In [ ]:
def get_priors(arms, n_features):
    posteriors = {}
    for arm in arms:
        m = np.zeros(n_features)
        s = np.ones(n_features)
        posteriors[arm] = [m, s]
    return posteriors

Функция для байесовской линейной регрессии

In [ ]:
def get_bandit_model(data, priors):
    ba_model = bmb.Model("reward ~ Age + ARPU", 
              data = data, 
              family="gaussian",
              priors={'Intercept':  bmb.Prior("Normal",mu= priors[0][0], sigma = priors[1][0]),
                            'Age': bmb.Prior("Normal",mu= priors[0][1], sigma = priors[1][1]),
                            'ARPU': bmb.Prior("Normal",mu= priors[0][2], sigma = priors[1][2]),
                            'reward_sigma':  bmb.Prior("Normal",mu= priors[0][3], sigma = priors[1][3])})
    trace = ba_model.fit(draws=2000,init='advi', target_accept=0.85, random_seed=42,inference_method='nuts_numpyro') #inference_method='nuts_numpyro'
    return trace  

Функция для семплирования Томпсона 

In [ ]:
def select_arm_thompson(posteriors, context, arms, alpha = 1):
    samples = {}
    context = np.insert(context, 0, 1)
    for arm in arms:
        m = posteriors[arm][0][:-1]
        s = posteriors[arm][1][:-1]* alpha
        w = np.random.normal(m, s)
        sample_prediction = np.dot(w, context)
        samples[arm] = sample_prediction
        
    max_value = max(samples.values()); 
    max_keys = [key for key, value in samples.items() if value == max_value]
    return np.random.choice(max_keys)

Запустим симуляцию

In [ ]:
cumulative_regret_list= []
cumulative_reward_list = []
total_regret = 0
cumulative_reward = 0
n_arms = 3
n_features = 4
arm_indices = np.array(range(n_arms))
posteriors = get_priors(arm_indices, n_features)
i = 0
X = {arm: [] for arm in arm_indices}
y = {arm: [] for arm in arm_indices}

In [ ]:
pbar = tqdm(desc = 'while loop', total = n)
while i != n:
    
    #generate a customer
    cust = get_customer()
    reward_vec = get_rewards(cust)
    #prepare features for model
    context = np.array([cust[1]])
        
                    
        # Display the selected arm
    if (i+1) < 500:
        arm = select_arm_thompson(posteriors, context, arm_indices,alpha=0.25)
    else:
        arm = select_arm_thompson(posteriors, context, arm_indices,alpha=0.25)
    i += 1
    
    reward = reward_vec[arm]
    X[arm].append(context.tolist()[0])
    y[arm].append(reward)
    
    best_choice = optimal_choices[cust[0]]
    mx = reward_vec[best_choice]
    
    regret = mx - reward
    total_regret += regret
    cumulative_regret_list.append(total_regret)
    cumulative_reward += reward
    cumulative_reward_list.append(cumulative_reward)
    pbar.update(1)
    #Update the models with the latest batch of data
    if (i + 1) % 500 == 0 and (i + 1) != n :
        print("Updating the models at i:", i + 1)
        print(f"Текущий regret: {cumulative_regret_list[-1]}")

        priors = posteriors
        posteriors = {}
        for arm in arm_indices:
            if X[arm] == [] or len(X[arm]) < 20:
                posteriors[arm] = priors[arm]
            else:
                
                train_df = pd.DataFrame(X[arm],
                            columns=["Age", 'ARPU'])
                train_df['reward'] = y[arm]
            
                trace = get_bandit_model(train_df, priors[arm])
            
                m = az.summary(trace)[['mean','sd']].values[:,0]
                s = az.summary(trace)[['mean','sd']].values[:,1] 
                                                
                posteriors[arm] = [m, s]
        X = {arm: [] for arm in arm_indices}
        y = {arm: [] for arm in arm_indices}
            
pbar.close()
df_cmab_regrets['PyMC_linear'] = cumulative_regret_list
df_cmab_rewards['PyMC_linear'] = cumulative_reward_list

### PYMC neural CMAB

Воспользуемся нашей реализацией нелинейного CMAB на сэмплировании Томпсона и нейронной сети

Определим функции для бандита

Зададим нейронную сеть

In [ ]:
# Standard PyTorch imports
import torch
from torch import nn
torch.manual_seed(42)
device = "cuda" if torch.cuda.is_available() else "cpu"
device

In [ ]:
class NN_model(nn.Module):
    def __init__(self):
        super().__init__()
        
        self.layer_1 = nn.Linear(in_features = 5, out_features=32)
        self.layer_2 = nn.Linear(in_features = 32, out_features=10)
        self.layer_3 = nn.Linear(in_features = 10, out_features=3)
        self.relu = nn.ReLU()
    

    def forward(self, x):
        
        x = self.relu(self.layer_1(x))
        x = self.relu(self.layer_2(x))
        x = self.layer_3(x)
        
        
        return x
                
    
    def get_representation(self, contexts):
        with torch.no_grad():
            x = self.relu(self.layer_1(contexts))
            x = self.relu(self.layer_2(x))
        return x

In [ ]:
def build_action_mask(actions, num_actions):
    ohe = torch.zeros((len(actions), 3))
    actions = actions.reshape(-1, 1)
    return ohe.scatter_(1, actions, 1)

In [ ]:
def build_target(rewards, actions, num_actions):
    
    ohe = torch.zeros((len(actions), num_actions))
    actions = actions.reshape(-1, 1)
    rewards = rewards.reshape(-1, 1)
    return ohe.scatter_(1, actions, rewards.float())

Зададим функцию для нашего бандита с фичами из нейронной сети

In [ ]:
def get_bandit_model(data, priors):
    ba_model = bmb.Model("reward ~ 0 + z1 + z2 + z3 + z4 +z5 + z6 + z7 + z8 + z9 + z10",
              data = data, 
              family="gaussian",
              priors={      'z1': bmb.Prior("Normal",mu= priors[0][0], sigma = priors[1][0]),
                            'z2': bmb.Prior("Normal",mu= priors[0][1] , sigma = priors[1][1]),
                            'z3': bmb.Prior("Normal",mu= priors[0][2], sigma = priors[1][2]),
                            'z4': bmb.Prior("Normal",mu= priors[0][3], sigma = priors[1][3]),
                            'z5': bmb.Prior("Normal",mu= priors[0][4], sigma = priors[1][4]),
                              'z6': bmb.Prior("Normal",mu= priors[0][5], sigma = priors[1][5]),
                              'z7': bmb.Prior("Normal",mu= priors[0][6], sigma = priors[1][6]),
                              'z8': bmb.Prior("Normal",mu= priors[0][7], sigma = priors[1][7]),
                              'z9': bmb.Prior("Normal",mu= priors[0][8], sigma = priors[1][8]),
                              'z10': bmb.Prior("Normal",mu= priors[0][9], sigma = priors[1][9]),
                            'reward_sigma':  bmb.Prior("Normal",mu= priors[0][10], sigma = priors[1][10])})
    trace = ba_model.fit(draws=3000,init='advi', target_accept=0.85, random_seed=42,inference_method='nuts_numpyro')
    return trace  

Функция для семплирования Томпсона

In [ ]:
def select_arm_thompson(model, posteriors, context, arms, alpha = 0.25):
    samples = {}
    for arm in arms:
        act = build_action_mask(torch.tensor([arm]), len(arms))
        context_nn = torch.cat((torch.tensor(context), act),1) 
        z = model.get_representation(context_nn.float()).detach().numpy()[0]
        m = posteriors[arm][0][:-1]
        s = posteriors[arm][1][:-1]* alpha
        w = np.random.normal(m, s)
        sample_prediction = np.dot(w, z)
        samples[arm] = sample_prediction
        
    max_value = max(samples.values()); 
    max_keys = [key for key, value in samples.items() if value == max_value]
    return np.random.choice(max_keys)

Веса для нейронки

In [ ]:
def init_weights(m):
    if isinstance(m, nn.Linear):
        torch.nn.init.uniform_(m.weight, a=-0.3, b=0.3)
        m.bias.data.fill_(0.01)

Запустим симуляцию

In [ ]:
cumulative_regret_list= []
cumulative_reward_list = []
total_regret = 0
cumulative_reward = 0
n_arms = 3
n_features = 11
epochs = 100
nn_model =  NN_model().to(device)
loss_fn = nn.MSELoss()
optimizer = torch.optim.RMSprop(nn_model.parameters(), lr=0.01)
scheduler1 = torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma=0.9)
scheduler2 = torch.optim.lr_scheduler.MultiStepLR(optimizer, milestones=[30,80], gamma=0.1)
nn_model = nn_model.apply(init_weights)


arm_indices = np.array(range(n_arms))
posteriors = get_priors(arm_indices, n_features)
i = 0
X = []
y = []
a = []
i = 0
features = ['z'+str(i) for i in np.arange(1,n_features)]
noise = np.random.normal(0, 0.0001, 500) # Случайный шум для обхода ошибки с константой

In [ ]:
pbar = tqdm(desc = 'while loop', total = n)
while i != n:
    
    #generate a customer
    cust = get_customer()
    reward_vec = get_rewards(cust)
    #prepare features for model
    context = np.array([cust[1]])
        
                    
        # Display the selected arm
    if (i) < 500:
        arm = np.random.choice(arm_indices)
    else:
        arm = select_arm_thompson(nn_model, posteriors, context, arm_indices)
    i += 1
    
    reward = reward_vec[arm]
    X.append(context.tolist()[0])
    y.append(reward)
    a.append(arm)
    
    best_choice = optimal_choices[cust[0]]
    mx = reward_vec[best_choice]
    regret = mx - reward
    total_regret += regret
    cumulative_regret_list.append(total_regret)
    cumulative_reward += reward
    cumulative_reward_list.append(cumulative_reward)
    pbar.update(1)
    #Update the models with the latest batch of data
    if i % 500 == 0 and  i != n:
        print("Updating the models at i:", i )
        
        batch_df = pd.DataFrame(X,
                            columns=["Age", 'ARPU'])
        batch_df['reward'] = y
        batch_df['actions'] = a
        print(batch_df.actions.value_counts())
        print(cumulative_regret_list[-1])

    
        w_batch = build_action_mask(torch.tensor(batch_df[['actions']].values), 3)
        X_batch = torch.cat((torch.tensor(batch_df[["Age", 'ARPU']].values), w_batch),1) 
        y_batch = build_target(torch.tensor(batch_df['reward'].values), torch.tensor(batch_df[['actions']].values), 3)
        print('Train Skynet')
        X_batch, y_batch, w_batch = X_batch.to(device), y_batch.to(device), w_batch.to(device)

        for epoch in tqdm(range(epochs)):
            
            for indx in np.arange(0,501,50):
                
                # 1. Forward pass
                y_pred = nn_model(X_batch[indx:indx+50].float())
                y_pred *= w_batch[indx:indx+50]

                # 2. Calculate loss and accuracy
                loss = loss_fn(y_pred, y_batch[indx:indx+50].float()) 


                # 3. Optimizer zero grad
                optimizer.zero_grad()

                # 4. Loss backward
                loss.backward()

                # 5. Optimizer step
                optimizer.step()
            scheduler1.step()
            scheduler2.step()

               
        
        z_values = nn_model.get_representation(X_batch.float())
        z_values = pd.DataFrame(z_values.detach().numpy())
        z_values.columns = features
        batch_df = pd.concat([batch_df, z_values],axis=1)
        batch_df[features] = batch_df[features].apply(lambda x: x + noise)
        
        print('Update bandit')
        
        priors = posteriors
        posteriors = {}
        for arm in arm_indices:
            if (batch_df.loc[batch_df['actions'] ==arm].empty) == True:
                posteriors[arm] = priors[arm]
            else:
                
                trace = get_bandit_model(batch_df.loc[batch_df['actions'] ==arm][['reward']+features], priors[arm])
            
                m = az.summary(trace)[['mean','sd']].values[:,0]
                s = az.summary(trace)[['mean','sd']].values[:,1]
                                                
                posteriors[arm] = [m, s]
        
        X = []
        y = []
        a = []
            
pbar.close()
df_cmab_regrets['PyMC_neural_linear'] = cumulative_regret_list
df_cmab_rewards['PyMC_neural_linear'] = cumulative_reward_list

### Посмотрим на бенчмарк 

In [ ]:
fig = px.line(df_cmab_rewards,labels={
                     "value": "Reward",
                    "index":'views',
                    "variable": "Approach"},)

fig.update_layout(title_text=f"""Кумулятивный reward""" ,showlegend=True)

fig.show() 

In [ ]:
fig = px.line(df_cmab_regrets,labels={
                     "value": "Regret",
                    "index":'views',
                    "variable": "Approach"},)

fig.update_layout(title_text=f"""Кумулятивный regret""" ,showlegend=True)

fig.show() 